In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!mkdir ./data 
!cp ./gdrive/MyDrive/Deep_Learning/data.zip  ./data/

In [3]:
!cd ./data && unzip data.zip

串流輸出內容已截斷至最後 5000 行。
  inflating: data/16703_right.jpeg   
  inflating: data/16697_left.jpeg    
  inflating: data/16684_left.jpeg    
  inflating: data/16641_right.jpeg   
  inflating: data/16548_right.jpeg   
  inflating: data/16528_right.jpeg   
  inflating: data/16479_left.jpeg    
  inflating: data/16391_right.jpeg   
  inflating: data/1638_right.jpeg    
  inflating: data/16289_right.jpeg   
  inflating: data/16236_left.jpeg    
  inflating: data/16224_right.jpeg   
  inflating: data/16203_right.jpeg   
  inflating: data/16145_left.jpeg    
  inflating: data/16142_right.jpeg   
  inflating: data/16131_right.jpeg   
  inflating: data/16129_right.jpeg   
  inflating: data/16114_right.jpeg   
  inflating: data/16109_left.jpeg    
  inflating: data/16101_left.jpeg    
  inflating: data/15944_left.jpeg    
  inflating: data/15808_left.jpeg    
  inflating: data/15540_left.jpeg    
  inflating: data/15522_left.jpeg    
  inflating: data/15505_left.jpeg    
  inflating: data/15471_right

## Load Data

In [11]:
import pandas as pd
from torch.utils import data
import numpy as np
from skimage import io, transform
import torchvision.transforms as transforms

import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score

In [5]:
def getData(mode):
    if mode == 'train':
        img = pd.read_csv('train_img.csv')
        label = pd.read_csv('train_label.csv')
        return np.squeeze(img.values), np.squeeze(label.values)
    else:
        img = pd.read_csv('test_img.csv')
        label = pd.read_csv('test_label.csv')
        return np.squeeze(img.values), np.squeeze(label.values)


In [41]:
class RetinopathyLoader(data.Dataset):
    def __init__(self, root, mode, transform=None):
        """
        Args:
            root (string): Root path of the dataset.
            mode : Indicate procedure status(training or testing)

            self.img_name (string list): String list that store all image names.
            self.label (int or float list): Numerical list that store all ground truth label values.
        """
        self.root = root
        self.img_name, self.label = getData(mode)
        self.mode = mode
        self.transform = transform
        print("> Found %d images..." % (len(self.img_name)))

    def __len__(self):
        """'return the size of dataset"""
        return len(self.img_name)

    def __getitem__(self, index):
        """something you should implement here"""

        """
           step1. Get the image path from 'self.img_name' and load it.
                  hint : path = root + self.img_name[index] + '.jpeg'
           
           step2. Get the ground truth label from self.label
                     
           step3. Transform the .jpeg rgb images during the training phase, such as resizing, random flipping, 
                  rotation, cropping, normalization etc. But at the beginning, I suggest you follow the hints. 
                       
                  In the testing phase, if you have a normalization process during the training phase, you only need 
                  to normalize the data. 
                  
                  hints : Convert the pixel value to [0, 1]
                          Transpose the image shape from [H, W, C] to [C, H, W]
                         
            step4. Return processed image and label
        """

        path_list = self.root + self.img_name[0:len(self.img_name)] + ".jpeg"
        
        image = io.imread(path_list[index])
        gt = self.label[index]
        if self.transform:
          image = self.transform(image)
        # image = np.transpose(image, (2, 0, 1))  
        

        img = image
        label = gt
        
        return img, label


In [42]:
input_root = "/content/data/data/"

train_transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.ToTensor()]) # calculte yourself

test_transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.ToTensor()]) # calculte yourself

train_dataset= RetinopathyLoader(input_root, "train", transform=train_transform)
train_dataloader = data.DataLoader(train_dataset, batch_size=10, shuffle=True)

test_dataset= RetinopathyLoader(input_root, "test", transform=test_transform)
test_dataloader = data.DataLoader(test_dataset, batch_size=10, shuffle=True)

> Found 28099 images...
> Found 7025 images...


In [7]:
# import matplotlib.pyplot as plt

# train_img, train_label = train_dataset[1]
# plt.imshow(np.transpose(train_img, (1, 2, 0)))
# print(train_img.shape, train_label.size)

In [44]:
class ResNet18_pretrain(nn.Module):
    def __init__(self):
        super(ResNet18_pretrain, self).__init__()

        # 載入 ResNet18 類神經網路結構
        self.model = models.resnet18(pretrained=True)
        
        # 修改輸出層輸出數量
        self.model.fc = nn.Linear(512, 5) # class = 5

    def forward(self, x):
        x = self.model(x)
        return x

## My Training 


In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [46]:
net = ResNet18_pretrain().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.0002)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [53]:
num_epochs = 10
num_dataloader = len(train_dataloader)

train_acc = np.zeros(num_epochs)
test_acc = np.zeros(num_epochs)

for epoch in range(num_epochs):
  correct = 0; total = 0;

  for i, (imgs, labels) in enumerate(train_dataloader):
    
    # print(imgs.shape)

    # zero the parameter gradients
    optimizer.zero_grad()

    # save training weight
    net.train()

    # forward + backward + optimize
    imgs = imgs.cuda()
    imgs = imgs.type(torch.cuda.FloatTensor)
    outputs = net(imgs)

    loss = criterion(outputs, labels.cuda())
    loss.backward()
    optimizer.step()

    if (i+1)%100==0:
      print("Epoch{}  {}/{} ".format(epoch+1, i+1, num_dataloader), "Training Loss = ", loss.item())

    for j, y in enumerate(outputs):
      y_list = y.tolist()
      kind = y_list.index(max(y.tolist()))
      if kind == labels[j]:
        correct += 1
    total += len(labels)

  
  train_acc[epoch] = correct/total
  print("Training Accuracy = ", correct/total)


  net.eval()

  # test
  test_correct = 0
  test_total = 0
  pred_test = np.zeros(len(test_dataset))
  for data in test_dataloader:
    test_imgs, test_labels = data
    # test_label = F.one_hot(torch.from_numpy(self.label).to(torch.int64)[index], num_classes=5)

    test_imgs = test_imgs.cuda()
    test_imgs = test_imgs.type(torch.cuda.FloatTensor)
    test_outputs = net(test_imgs)

    # kind = np.zeros(len(test_labels)) #batch size
    for j, y in enumerate(test_outputs):
      y_list = y.tolist()
      kind = y_list.index(max(y.tolist()))
      if kind == test_labels[j]:
        test_correct += 1
    test_total += len(test_labels)

  
  test_acc[epoch] = test_correct/test_total
  print("Test Accuracy = ", test_correct/test_total)

  save_path = '/content/gdrive/MyDrive/Deep_Learning/model_18_T_' + str(test_acc[epoch]) + '.pth'
  torch.save(net, save_path)


Epoch1  100/2810  Training Loss =  0.8211696743965149
Epoch1  200/2810  Training Loss =  0.6107217073440552
Epoch1  300/2810  Training Loss =  1.4216439723968506
Epoch1  400/2810  Training Loss =  0.7918580770492554
Epoch1  500/2810  Training Loss =  0.806158185005188
Epoch1  600/2810  Training Loss =  0.8630698919296265
Epoch1  700/2810  Training Loss =  1.0527756214141846
Epoch1  800/2810  Training Loss =  1.1473442316055298
Epoch1  900/2810  Training Loss =  0.9629939198493958
Epoch1  1000/2810  Training Loss =  0.8561722040176392
Epoch1  1100/2810  Training Loss =  1.0351097583770752
Epoch1  1200/2810  Training Loss =  0.7826465964317322
Epoch1  1300/2810  Training Loss =  0.476546049118042
Epoch1  1400/2810  Training Loss =  0.6952512264251709
Epoch1  1500/2810  Training Loss =  1.383813500404358
Epoch1  1600/2810  Training Loss =  0.5518649220466614
Epoch1  1700/2810  Training Loss =  0.983599841594696
Epoch1  1800/2810  Training Loss =  0.6537833213806152
Epoch1  1900/2810  Trai

In [54]:
from google.colab import files

np.savez('DL_lab3_ResNet18_T_acc.npz', train=train_acc, test=test_acc)
files.download('DL_lab3_ResNet18_T_acc.npz') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## plot

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
cm = confusion_matrix(test_pred_array, test_label_array, normalize='all')

In [ ]:
a = ConfusionMatrixDisplay(cm)
a.plot()